In [ ]:
## try download button

## tutorial of a clean working example: https://community.plotly.com/t/download-raw-data/4700/49

import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import urllib
#from urllib.parse import urlparse
#import urllib.parse
#urllib.parse.quote
#from urllib.quote import quote

df = pd.DataFrame({
    'a': [1, 2, 3, 4],
    'b': [2, 1, 5, 6],
    'c': ['x', 'x', 'y', 'y']
})


def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )


app = dash.Dash(__name__)

#app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})
app.layout = html.Div([
    html.Label('Filter'),

    dcc.Dropdown(
        id='field-dropdown',
        options=[
            {'label': i, 'value': i} for i in
            (['all'] + list(df['c'].unique()))],
        value='all'
    ),
    html.Div(id='table'),
    html.A(
        'Download Data',
        id='download-link',
        download="rawdata.csv",
        href="",
        target="_blank"
    )
])


def filter_data(value):
    if value == 'all':
        return df
    else:
        return df[df['c'] == value]


@app.callback(
    dash.dependencies.Output('table', 'children'),
    [dash.dependencies.Input('field-dropdown', 'value')])
def update_table(filter_value):
    dff = filter_data(filter_value)
    return generate_table(dff)


@app.callback(
    dash.dependencies.Output('download-link', 'href'),
    [dash.dependencies.Input('field-dropdown', 'value')])
def update_download_link(filter_value):
    dff = filter_data(filter_value)
    csv_string = dff.to_csv(index=False, encoding='utf-8')
    csv_string = "data:text/csv;charset=utf-8,%EF%BB%BF" + urllib.parse.quote(csv_string)
    return csv_string


# if __name__ == '__main__':
#     app.run_server(debug=True)```

app.run_server(debug=True, use_reloader=False)

In [ ]:
# try out
#checkpoint: Jun2 11:12am works!

import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

df = px.data.gapminder()

app = dash.Dash()
app.layout = html.Div([
    
    html.H1("Hello World"),
    
    dcc.Markdown('''
    Trying out paragraph writing
    '''),
    
    dcc.Graph(id = 'gapminder-graph'), # figure=fig
    
    dcc.Dropdown(
                id='continent-name',
                options=[{'label': i, 'value': i} for i in df.continent.unique()],
                value='Oceania' # set a default value
            ),
    
    html.H2("datatable"),
    
    html.A(
        'Download Data',
        id='download-link',
        download="rawdata.csv",
        href="",
        target="_blank"
    ),
    
    dash_table.DataTable(
    id='table',
    #columns=[{"name": i, "id": i} for i in df.columns],
    columns=[{"name": i, "id": i} for i in ['country','year','lifeExp']] #specifying column names here won't affect fieldnames in csvfile dl
    #data=df[df['continent']=='Asia'].to_dict('records'), #passed in a filtering here
)
])


@app.callback(
         Output('gapminder-graph','figure'),
          [Input('continent-name','value')])
def update_graph(continent):
    
    dff = df[df['continent']==continent]
    fig = px.line(dff, x="year", y="lifeExp", color='country')
    
    return fig

@app.callback(
    Output('table','data'),
    [Input('continent-name','value')])
def update_table(continent):
    data=df[df['continent']==continent].to_dict('records')
    return data

@app.callback(
    Output('download-link', 'href'),
    [Input('continent-name', 'value')])
def update_download_link(filter_value):
    dff = df[df['continent']==filter_value]
    csv_string = dff.to_csv(index=False, encoding='utf-8')
    csv_string = "data:text/csv;charset=utf-8,%EF%BB%BF" + urllib.parse.quote(csv_string)
    return csv_string
    

app.run_server(debug=True, use_reloader=False)